#  <a id='class'> Classificaton  </a>
### <a id='setup'>Setup:</a>

In [ ]:
from scipy.optimize import curve_fit

###  <a id='fit'>Curve fitting: </a>

In [ ]:
def fit_curves(time_readings, pressure_readings, mask, verbose=False):
    def poly(x, a,b,c,d):
        y =  a*(x**3)+b*(x**2)+c*(x) + d
        return y

    def decay(x,a,b,c):
        y = a*(1 - x)**b + c
        return y

    skipP,skipE = False,False
    try:
        popt1, pcov1 = curve_fit(poly, time_readings[mask], pressure_readings[mask])
        poly_fit = poly(time_readings[mask], *popt1)
    except:
        print("Could not fit polynomial")
        skipP = True
    try:
        popt2, pcov2 = curve_fit(decay, time_readings[mask], pressure_readings[mask])
        decay_fit = decay(time_readings[mask],*popt2)
    except:
        print("Could not fit exponential")
        skipE = True

    if not skipP:
        msq_err1 = np.mean((pressure_readings[mask]-poly_fit)**2)
        if verbose:
            print("Polynomial fit: y = %.3fx^3 + %.3fx^2 + %.3fx + %.3f"%(popt1[0],popt1[1],popt1[2],popt1[3]))
            print("MeanSqr Error Polynomial:%.4f"%(msq_err1))
    if not skipE:
        msq_err2 = np.mean((pressure_readings[mask]-decay_fit)**2)
        if verbose:
            print("Decay fit: y = %.3f(1-x)^%.3f + %.3f"%(popt2[0],popt2[1],popt2[2]))
            print("MeanSqr Error Exponential:%.4f"%(msq_err2))
    
    #Polynomial mean square error must be at least 10% better
    coupled = None
    if msq_err2 > msq_err1*1.5:
        coupled = 1
        print("Coupling Detected! \nDifference in errors %.4f"%(abs(msq_err2-msq_err1)))
    else:
        coupled = 0
        print("Normal Gauge behaviour")
    return poly_fit,decay_fit, coupled

### <a id='allin1'> All-In-One Curve Fitting Classifier Function </a>

In [ ]:
def analytical_classifier(gauge_id, fillNumber=2216, showPlot=True, verbose=False, use_mask = True):
    # Data Retrieval
    pressure_readings,time_readings,beam_time,beam_energy = retrieve_gauge_data(gauge_id, fillNumber)
    # Data Normalization
    pressure_readings,beam_energy = normalize_y(time_readings, beam_time, pressure_readings, beam_energy)
    # Data Interpolation
    time_readings,pressure_readings,beam_time,beam_energy= interpolate_readings(pressure_readings, time_readings, beam_time, beam_energy)
    
    # Data Masking
    if use_mask:
        mask,threshold = double_threshold_energy_masking(time_readings,beam_time,beam_energy)
    else:
        mask = np.arange(0,len(pressure_readings),1)
    
    # Forward Fourier Transform
    pressure_transform, spectrum,deltaT = forward_fourier_transform(time_readings,pressure_readings)
    # Constrained Inverse Fourier Transform
    time_constrained, signal_constrained = filtered_inverse_fourier_transform(pressure_transform,
                                                                              deltaT, spectrum[0], 40)


    # Curve Fitting
    fit1,fit2, coupled = fit_curves(time_constrained,signal_constrained,mask,verbose=verbose)
    # Plotting
    f, ax = plot_analytical_classifier(time_readings, pressure_readings, time_constrained, signal_constrained, fit1, fit2, mask)
    if showPlot:
        plt.show(f)
        
    return f, ax, coupled

# Analytical Classifier \[Gradient\]

In [1]:
def VG_analyzer_simple(gauge_id,fillNumber,cuts=10, plot=True, all_plots=False):
    # Data Retrieval
    pressure_readings,time_readings,beam_time,beam_energy = retrieve_gauge_data(gauge_id,fillNumber, verbose=False,
                                                                               show_plot=False)
    # Data Normalization
    pressure_readings,beam_energy = normalize_y(time_readings, beam_time, pressure_readings, beam_energy,
                                        show_plot=False)
    # Data Interpolation
    time_readings,pressure_readings,beam_time,beam_energy= interpolate_readings(pressure_readings, time_readings, beam_time,
                                                                                beam_energy, show_plot=False)
    # Forward Fourier Transform
    pressure_transform, spectrum,deltaT = forward_fourier_transform(time_readings,pressure_readings,
                                                                   show_plot=False)
    # Constrained Inverse Fourier Transform
    time_constrained, signal_constrained = filtered_inverse_fourier_transform(pressure_transform,
                                                                              deltaT,spectrum[0],40,
                                                                             show_plot=False)
    # Data Masking
    mask,threshold = double_threshold_energy_masking(time_readings,beam_time,beam_energy,
                                                     show_plot=False)
    
    levels = level_data(signal_constrained[mask], cuts=cuts)
    
    pressure_1st_deriv = np.diff(levels, n=1) #1st order
    pressure_2nd_deriv = np.diff(levels, n=2) #2nd order
    
    if plot:
        f, ax = plot_levelled_data(gauge_id, levels, time_readings,pressure_readings, show=plot)
    else:
        f, ax = None, None
        
    # If overall gradient is negative ==> expected behaviour
    if np.all(pressure_1st_deriv<0):
        print("Expected Behaviour")
        return f,ax,0
    else:
        print("Unexpected Behavour")
        return f,ax,1